In [1]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico, options=options)

In [2]:
# abrir a página index (entrar no site da busca jurídica)
import os
caminho = os.getcwd()
arquivo = caminho + r'\Sites\index2.html'
navegador.get(arquivo)


In [3]:
import pandas as pd

tabela = pd.read_excel('Excel\Processos.xlsx')
display(tabela)


Nome      Advogado Processo            Cidade  Status
0    Lira   Alon Lawyer   PC6592  Distrito Federal     NaN
1    João   Lawyer Alon   EB3792    Rio de Janeiro     NaN
2  Amanda  Amanda mesmo   MM1043    Rio de Janeiro     NaN
3   Carol        Amanda   PC5197         São Paulo     NaN

In [4]:
from selenium.webdriver import ActionChains
from selenium.webdriver.common.alert import Alert

for linha in tabela.index:
    #para cada processo (linha da tabela)
    navegador.get(arquivo)

    # passando o mouse pelo botão
    botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
    ActionChains(navegador).move_to_element(botao).perform()
    
    #clicando no estado
    cidade = tabela.loc[linha, 'Cidade']
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    # trocar de uma janela para outra
    aba_estado = navegador.window_handles[0]
    indice = 1 + linha
    aba_pesquisa = navegador.window_handles[indice]
    navegador.switch_to.window(aba_pesquisa)
    
    # completar pesquisa jurídica
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, 'Nome']) #nome
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, 'Advogado']) #advogado
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, 'Processo']) #Nº processos

    #clicar no botão de pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
    
    # lidando com os prompts


    alerta = Alert(navegador)
    alerta.accept()
    
    # esperar o alertar aparecer, e depos aceita-lo

    while True:
        try:
            alerta = Alert(navegador)
            texto_alerta = alerta.text
            break
        except:
            time.sleep(1)
            
    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela.loc[linha, 'Status'] = "Encontrado"
    else:
        tabela.loc[linha, 'Status'] = "Não encontrado"
        alerta.accept()

In [5]:
navegador.quit()
display(tabela)

Nome      Advogado Processo            Cidade          Status
0    Lira   Alon Lawyer   PC6592  Distrito Federal  Não encontrado
1    João   Lawyer Alon   EB3792    Rio de Janeiro  Não encontrado
2  Amanda  Amanda mesmo   MM1043    Rio de Janeiro      Encontrado
3   Carol        Amanda   PC5197         São Paulo  Não encontrado